In [1]:
import os
from typing import List
import json

import requests as requests
from typing import Union

from web3 import Web3, HTTPProvider, IPCProvider

In [2]:
ALCHEMY_API_KEY = os.environ['ALCHEMY_API_KEY']
web3 = Web3(HTTPProvider(f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}"))

In [3]:
def alchemy_asset_transfers(
        from_address: str,
        to_address: str,
        from_block: Union[int, str],
        to_block: Union[int, str] = 'latest',
        page_key: str = None
):
    if not to_block == 'latest':
        to_block = hex(to_block)

    parameters = {
        "fromBlock": hex(from_block),
        "toBlock": to_block,
        "fromAddress": from_address,
        "toAddress": to_address
    }

    if not from_address:
        parameters.pop('fromAddress')
    if not to_address:
        parameters.pop('toAddress')

    if page_key:
        parameters['pageKey'] = page_key

    response_post = requests.post(
        "https://eth-mainnet.alchemyapi.io/v2/"
        + os.environ["ALCHEMY_API_KEY"],
        json={
            "jsonrpc": "2.0",
            "id": 0,
            "method": "alchemy_getAssetTransfers",
            "params": [parameters]
        },
    )

    response_json = response_post.json()
    return response_json

def get_lp_swaps(
        from_address: str,
        to_address: str,
        from_block: Union[int, str],
        to_block: Union[int, str]
) -> List:

    page_key = None
    all_txes = []
    while True:
        response = alchemy_asset_transfers(
            from_address=from_address,
            to_address=to_address,
            from_block=from_block,
            to_block=to_block,
            page_key=page_key
        )
        transactions = response["result"]["transfers"]
        all_txes.extend(transactions)
        if not 'pageKey' in response['result'].keys():
            break
        page_key = response['result']['pageKey']

    return all_txes

In [4]:
pool_addr = "0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7"
with open("./3pool.json", 'r') as f:
    abi = json.load(f)
contract = web3.eth.contract(address=pool_addr, abi=abi)

In [5]:
fraction_fee = 0.03 / 100

In [6]:
block_ranges = [(12693255, 12733780),
                (12733780, 12740342),
                (12740342, 12826286),
                (12826286, 12865685),
                (12865685, 12960904),
                (12960904, 12993519),
                (12993519, 13048460),
                (13048460, 13073420),
                (13073420, 13182518),
                (13182518, 13206754),
                (13206754, 13278344),
                (13278344, 13307602),
                (13307602, 13360861),
                (13360861, 13402155),
                (13402155, 13456217),
                (13456217, 13484749),
                (13484749, 13541142)]

In [7]:
fees_for_range = {}
for (block_start, block_end) in block_ranges:

    from_txes = get_lp_swaps(
        from_address=pool_addr, to_address='', from_block=block_start, to_block=block_end
    )
    from_tx_hashes = [i['hash'] for i in from_txes]

    to_txes = get_lp_swaps(
        from_address='', to_address=pool_addr, from_block=block_start, to_block=block_end
    )
    to_tx_hashes = [i['hash'] for i in to_txes]

    total_fees = 0
    for i in to_txes:
        # add_liquidity txes and swaps
        out_value = i['value']
        total_fees += i['value'] * fraction_fee

    # remove liquidity imbalanced:
    event_filter = contract.events.RemoveLiquidityImbalance.createFilter(
        fromBlock=block_start,
        toBlock=block_end
    )
    remove_liquidity_imbalanced = event_filter.get_all_entries()
    remove_liquidity_imbalanced_tx_hash = []
    for event in remove_liquidity_imbalanced:
        fees = event['args']['fees']
        total_fees += fees[0]*1e-18 + 1e-6*(fees[1] + fees[2])
        remove_liquidity_imbalanced_tx_hash.append(event['blockHash'].hex())

    # remove liquidity one:
    for i in from_txes:
        # if not a swap and not a tx with remove liquidity imbalance event
        if i not in [to_txes + remove_liquidity_imbalanced_tx_hash]:
            out_value = i['value']
            in_value = out_value/(1-fraction_fee)
            estimated_fees = fraction_fee*in_value
            total_fees += estimated_fees

    fees_for_range[(block_start, block_end)] = total_fees

In [8]:
fees_for_range

{(12693255, 12733780): 307410.7348346571,
 (12733780, 12740342): 103771.17515825875,
 (12740342, 12826286): 949705.8357643525,
 (12826286, 12865685): 508285.0047144548,
 (12865685, 12960904): 1059108.6911517421,
 (12960904, 12993519): 467215.7981490432,
 (12993519, 13048460): 751542.6233646222,
 (13048460, 13073420): 385974.7236686516,
 (13073420, 13182518): 1166879.5080928653,
 (13182518, 13206754): 369635.16680853924,
 (13206754, 13278344): 866113.3254025063,
 (13278344, 13307602): 416406.7956135295,
 (13307602, 13360861): 429318.25855426985,
 (13360861, 13402155): 726002.6644070608,
 (13402155, 13456217): 1062627.5215839637,
 (13456217, 13484749): 382623.9428388932,
 (13484749, 13541142): 802364.887326217}